In [1]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import tqdm

In [2]:
#header for metadata
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [3]:
url2 = "https://housing.com/in/buy/hyderabad/hyderabad"

In [4]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
page = requests.get(url2, headers=headers)#, proxies=proxies)
content = page.content
soup = BeautifulSoup(content)
places_list = []
for tag in soup.find_all('a'):
  link = tag.get("href")
  searchd = "/in/buy/hyderabad"
  if link.find(searchd) != -1:
    new_link = "https://housing.com" + link
    places_list.append(new_link)
print(places_list)
Total_list1=[]
for i in places_list:
  page = requests.get(i, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  link_list = []
  for tag in soup.find_all('a'):
    link = tag.get("href")
    searchd = "/in/buy/projects/page"
    if link.find(searchd) != -1:
      new_link = "https://housing.com" + link
      link_list.append(new_link)
  Total_list1.extend(link_list)
print(Total_list1)

['https://housing.com/in/buy/hyderabad/kismatpur', 'https://housing.com/in/buy/hyderabad/sangareddy', 'https://housing.com/in/buy/hyderabad/shankarpally', 'https://housing.com/in/buy/hyderabad/balapur', 'https://housing.com/in/buy/hyderabad/kollur', 'https://housing.com/in/buy/hyderabad/bowrampet', 'https://housing.com/in/buy/hyderabad/narapally', 'https://housing.com/in/buy/hyderabad/kompally', 'https://housing.com/in/buy/hyderabad/shadnagar', 'https://housing.com/in/buy/hyderabad/yapral', 'https://housing.com/in/buy/hyderabad/hafeezpet', 'https://housing.com/in/buy/hyderabad/hitech_city', 'https://housing.com/in/buy/hyderabad/osman_nagar_hyderabad', 'https://housing.com/in/buy/hyderabad/bongloor', 'https://housing.com/in/buy/hyderabad/premavathi_pet', 'https://housing.com/in/buy/hyderabad/bhadurpalle', 'https://housing.com/in/buy/hyderabad/nanakram_guda', 'https://housing.com/in/buy/hyderabad/kollur', 'https://housing.com/in/buy/hyderabad/dundigal', 'https://housing.com/in/buy/hydera

In [6]:
def extract_data(url):
  word="Sizes.css-ltuii1::after{content:'\\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"
  page = requests.get(url, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  #print(soup)
  result = {}
  flat_data = soup.findAll('div',attrs={'class' : 'css-j2zgcq'})
  flat_name = flat_data[0].find('h1',attrs={'class':'css-1hidc9c'})
  if flat_name is not None:
    flat_name = flat_name.text
  else:
    flat_name = "Flat name not mentioned"
  result['Flat Name'] = flat_name
  flat_address = flat_data[0].find('div',attrs={'class':'css-1ty5xzi'})
  if flat_address is not None:
    flat_address = flat_address.text
  else:
    flat_address = "Flat address not mentioned"
  result['Flat address'] = flat_address
  key = []
  value = []
  word2="Sizes"
  for data in soup.findAll('section',attrs={'class' : 'css-13dph6'}):
    for i in data.findAll('div',attrs={'class' : 'css-0'}):
      if i.text == word:
        key.append(word2)
      else:
        key.append(i.text)
    for j in data.findAll('div',attrs={'class' : 'css-1k19e3'}):
      value.append(j.text)
  for i in range(0,len(value)):
    result[key[i]] = value[i]
  tables = soup.find_all("table")
  table = tables[0]
  key_data=[]
  value_data=[]
  #tab_data = [[cell.text for cell in row.find_all(["th","td"])] for row in table.find_all("tr")]
  for row in table.find_all("tr"):
    for cell in row.find_all("th"):
      key_data.append(cell.text)
    for cell in row.find_all("td"):
      for data in cell.find_all("div"):
        value_data.append(data.text)
  for i in range(0,len(value_data)-1):
    result[key_data[i]] = value_data[i]
  data = soup.findAll("div",attrs={"class":"css-1vyxx8y"})
  for i in data:
    result["About"]=i.text
  images = soup.find_all('img')
  images_list = []
  for image in images: 
    if image['src'].startswith("https://"):
      images_list.append(image['src'])
  result["images"]=images_list
  return result

In [7]:
total_dict2=[]
for i in range(0,110):
  print(Total_list1[i])
  total_dict2.append(extract_data(Total_list1[i]))
print(total_dict2)

https://housing.com/in/buy/projects/page/53162-giridhari-prospera-county-by-giridhari-constructions-in-kismatpur
https://housing.com/in/buy/projects/page/32365-prestige-royal-woods-by-prestige-estates-projects-ltd-in-kismatpur
https://housing.com/in/buy/projects/page/53162-giridhari-prospera-county-by-giridhari-constructions-in-kismatpur?fltcnt=2b93b3ed-fbd2-42d9-93ed-1e0593d5873f
https://housing.com/in/buy/projects/page/263432-vaishnavi-houdini-by-vaishnavi-infracon-india-pvt-ltd-in-bandlaguda-jagir?fltcnt=c1c02b6f-1081-4cd8-837a-0ed071e88825
https://housing.com/in/buy/projects/page/30274-shantasriram-brookwoods-by-shanta-sriram-constructions-in-kismatpur
https://housing.com/in/buy/projects/page/86471-provident-manhattan-pods-and-condos-by-provident-housing-limited-in-premavathi-pet
https://housing.com/in/buy/projects/page/53161-vaishnavi-triumph-villas-by-vaishnavi-estates-in-darga-khaliz-khan
https://housing.com/in/buy/projects/page/32365-prestige-royal-woods-by-prestige-estates-pro

In [8]:
data = pd.DataFrame(total_dict2)
data.head(10)

,Flat Name,Flat address,Configurations,Possession Starts,Avg. Price,Sizes,Project Size,Launch Date,About,images,Possession Status,Project Area,Configuration,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size
0,Giridhari Prospera County,"Rajendra Nagar Road, Kismatpur, West Hyderabad...","4, 4.5 BHK Villa, Triplexes","Dec, 2023",14.00 K/sq.ft,4750.00 sq.ft. - 6150.00 sq.ft.,98 units,"Jun, 2020",Giridhari Constructions present a premier hous...,[https://is1-2.housingcdn.com/4f2250e8/c7d7672...,NaN,NaN,NaN,NaN,NaN
1,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",NaN,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN
2,Giridhari Prospera County,"Rajendra Nagar Road, Kismatpur, West Hyderabad...","4, 4.5 BHK Villa, Triplexes","Dec, 2023",14.00 K/sq.ft,4750.00 sq.ft. - 6150.00 sq.ft.,98 units,"Jun, 2020",Giridhari Constructions present a premier hous...,[https://is1-2.housingcdn.com/4f2250e8/c7d7672...,NaN,NaN,NaN,NaN,NaN
3,Vaishnavi Houdini,"Bandlaguda Jagir, West Hyderabad, Hyderabad","2, 3 BHK Apartments","Dec, 2024",5.20 K/sq.ft,1193.00 sq.ft. - 2028.00 sq.ft.,3 Buildings - 585 units,"Jun, 2021",For those who dream with a passion for the ext...,[https://is1-2.housingcdn.com/4f2250e8/f6d0335...,NaN,4.11 Acres,NaN,NaN,NaN
4,Shantasriram Brookwoods,"APPA Junction, Near Masqati Dairy Product Kism...","3, 4 BHK Villas","Apr, 2021",7.50 K/sq.ft,3356.00 sq.ft. - 3814.00 sq.ft.,49 units,"Dec, 2018",Shantasriram Brookwoods is a residential proje...,[https://is1-2.housingcdn.com/4f2250e8/126b607...,NaN,NaN,NaN,NaN,NaN
5,Provident Manhattan Pods And Condos,"Rajendra Nagar., West Hyderabad, Hyderabad","1 RK, 1 BHK Apartments","Oct, 2021",Price on request,380.00 sq.ft. - 610.00 sq.ft.,200 units,"Feb, 2018","Situated in Hyderabad at Premavathi Pet, Provi...",[https://is1-3.housingcdn.com/4f2250e8/2d9acba...,NaN,NaN,NaN,NaN,NaN
6,Vaishnavi Triumph Villas,"Kismatpur, West Hyderabad, Hyderabad",NaN,"Mar, 2020",7.00 K/sq.ft,3264.00 sq.ft. - 3661.00 sq.ft.,109 units,"Dec, 2017",Vaishnavi Estates presents villas of alluring ...,[https://is1-3.housingcdn.com/4f2250e8/006fe35...,NaN,11 Acres,3 BHK Triplex,NaN,NaN
7,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",NaN,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN
8,Provident Manhattan Pods And Condos,"Rajendra Nagar., West Hyderabad, Hyderabad","1 RK, 1 BHK Apartments","Oct, 2021",Price on request,380.00 sq.ft. - 610.00 sq.ft.,200 units,"Feb, 2018","Situated in Hyderabad at Premavathi Pet, Provi...",[https://is1-3.housingcdn.com/4f2250e8/2d9acba...,NaN,NaN,NaN,NaN,NaN
9,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",NaN,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN


In [9]:
data["Configurations"]=data['Configurations'].combine_first(data['Configuration'])

In [10]:
data["Sizes"]=data['Sizes'].combine_first(data['Size'])

In [11]:
data=data.rename(columns = {'Possession Starts':'Possession_Starts','Possession Status':'Possession_Status'})

In [12]:
data["Possession_Starts"]=data['Possession_Starts'].combine_first(data['Possession_Status'])

In [13]:
data.head(10)

,Flat Name,Flat address,Configurations,Possession_Starts,Avg. Price,Sizes,Project Size,Launch Date,About,images,Possession_Status,Project Area,Configuration,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size
0,Giridhari Prospera County,"Rajendra Nagar Road, Kismatpur, West Hyderabad...","4, 4.5 BHK Villa, Triplexes","Dec, 2023",14.00 K/sq.ft,4750.00 sq.ft. - 6150.00 sq.ft.,98 units,"Jun, 2020",Giridhari Constructions present a premier hous...,[https://is1-2.housingcdn.com/4f2250e8/c7d7672...,NaN,NaN,NaN,NaN,NaN
1,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",Ready to Move in,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN
2,Giridhari Prospera County,"Rajendra Nagar Road, Kismatpur, West Hyderabad...","4, 4.5 BHK Villa, Triplexes","Dec, 2023",14.00 K/sq.ft,4750.00 sq.ft. - 6150.00 sq.ft.,98 units,"Jun, 2020",Giridhari Constructions present a premier hous...,[https://is1-2.housingcdn.com/4f2250e8/c7d7672...,NaN,NaN,NaN,NaN,NaN
3,Vaishnavi Houdini,"Bandlaguda Jagir, West Hyderabad, Hyderabad","2, 3 BHK Apartments","Dec, 2024",5.20 K/sq.ft,1193.00 sq.ft. - 2028.00 sq.ft.,3 Buildings - 585 units,"Jun, 2021",For those who dream with a passion for the ext...,[https://is1-2.housingcdn.com/4f2250e8/f6d0335...,NaN,4.11 Acres,NaN,NaN,NaN
4,Shantasriram Brookwoods,"APPA Junction, Near Masqati Dairy Product Kism...","3, 4 BHK Villas","Apr, 2021",7.50 K/sq.ft,3356.00 sq.ft. - 3814.00 sq.ft.,49 units,"Dec, 2018",Shantasriram Brookwoods is a residential proje...,[https://is1-2.housingcdn.com/4f2250e8/126b607...,NaN,NaN,NaN,NaN,NaN
5,Provident Manhattan Pods And Condos,"Rajendra Nagar., West Hyderabad, Hyderabad","1 RK, 1 BHK Apartments","Oct, 2021",Price on request,380.00 sq.ft. - 610.00 sq.ft.,200 units,"Feb, 2018","Situated in Hyderabad at Premavathi Pet, Provi...",[https://is1-3.housingcdn.com/4f2250e8/2d9acba...,NaN,NaN,NaN,NaN,NaN
6,Vaishnavi Triumph Villas,"Kismatpur, West Hyderabad, Hyderabad",3 BHK Triplex,"Mar, 2020",7.00 K/sq.ft,3264.00 sq.ft. - 3661.00 sq.ft.,109 units,"Dec, 2017",Vaishnavi Estates presents villas of alluring ...,[https://is1-3.housingcdn.com/4f2250e8/006fe35...,NaN,11 Acres,3 BHK Triplex,NaN,NaN
7,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",Ready to Move in,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN
8,Provident Manhattan Pods And Condos,"Rajendra Nagar., West Hyderabad, Hyderabad","1 RK, 1 BHK Apartments","Oct, 2021",Price on request,380.00 sq.ft. - 610.00 sq.ft.,200 units,"Feb, 2018","Situated in Hyderabad at Premavathi Pet, Provi...",[https://is1-3.housingcdn.com/4f2250e8/2d9acba...,NaN,NaN,NaN,NaN,NaN
9,Prestige Royal Woods,"Kismatpur, West Hyderabad, Hyderabad","3, 3.5, 4.5 BHK Villas",Ready to Move in,9.30 K/sq.ft,2838.00 sq.ft. - 5200.00 sq.ft.,152 units,"Feb, 2011",Prestige Royal Woods comes up as an exclusive ...,[https://is1-2.housingcdn.com/4f2250e8/668126d...,Ready to Move in,24 Acres,NaN,NaN,NaN


In [15]:
data = data.drop(columns=['Configuration', 'Possession_Status','Size',"Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"])

In [16]:
data.to_csv('Hyderabad.csv', index=True)